In [1]:
from lion_weapon import LionWeapon


lp = LionWeapon()
lp.calculate()

  1 of 60 documents processed.
        data compra_venda abbreviation titulo ativo    qtd  preco  \
0 2017-03-17            C        JBSS3    JBS    ON  200.0  11.03   
1 2017-03-17            C        JBSS3    JBS    ON  300.0  11.19   

   valor_operacao  taxas  total_ajustado  preco_ajustado  
0          2206.0  21.13         2214.45       11.072250  
1          3357.0  21.13         3369.68       11.232267  
              preco_medio     qtd
abbreviation                     
BBDC4            5.643477   586.0
JBSS3           11.168260   500.0
PSSA3           12.372676  2700.0
WEGE3           20.324935  1000.0


In [4]:
raw_df = lp.raw_df

pos = raw_df[(raw_df.abbreviation=='PSSA3') & (raw_df.compra_venda == 'C')].sort_values("data")
neg = raw_df[(raw_df.abbreviation=='PSSA3') & (raw_df.compra_venda == 'V')].sort_values("data")
# calc.total_ajustado.sum()
total = pos.total_ajustado.sum() - neg.total_ajustado.sum()
total

AttributeError: 'NoneType' object has no attribute 'raw_df'

In [ ]:
raw_df.data